<a href="https://colab.research.google.com/github/irika-katiyar/irika-katiyar-twitter/blob/main/CAISProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#imports / mounting to dive
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/CAIS Project

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/CAIS Project


In [8]:
!pip install keras

In [29]:
#df = pd.read_csv("/content/drive/My Drive/CAIS Project/dataset/data.csv", header=0, skiprows=[i for i in range(1,795000)], nrows=10000)
df = pd.read_csv("/content/drive/My Drive/CAIS Project/dataset/data-small.csv", header=0)

#using GloVe file from Lesson 6 RNN example to avoid having to download and put into my Google Drive
EMBEDDINGS_DIR = '/content/drive/My Drive/2021 Fall Curriculum/Lessons/Lesson 6: RNNs/glove.6B.50d.txt'

#data preprocessing using method similar to Lesson 6 RNN example
%run -i load_data.py
tweets, tweets_preprocessed, labels, word_index, embedding_matrix = load_data(df, EMBEDDINGS_DIR)


1: Setting tweets and labels to columns of df
2: tokenizing tweets
3: padding
4: loading word embeddings
5: finding word embeddings


In [30]:
#seeing what training data looks like
print("Training Data Size: ", tweets_preprocessed.shape)
print("Number of Tweets: ", tweets_preprocessed.shape[0])
print("Max Tweet Length: ", tweets_preprocessed.shape[1])

Training Data Size:  (268012, 46)
Number of Tweets:  268012
Max Tweet Length:  46


In [31]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras import metrics
from keras.models import Model
import pickle

In [32]:
#creating model using 2 LSTM layers and 2 Dense layers
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=tweets_preprocessed.shape[1],
                    trainable=False))
model.add(LSTM(64, return_sequences = True, activation='relu'))
model.add(Dropout(.2))

model.add(LSTM(64, activation='relu'))
model.add(Dropout(.2))

model.add(Dense(32, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

print(model.summary())

model.compile(loss = 'binary_crossentropy', optimizer = 'RMSprop', metrics=[metrics.binary_accuracy])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 46, 50)            9794000   
                                                                 
 lstm_12 (LSTM)              (None, 46, 64)            29440     
                                                                 
 dropout_12 (Dropout)        (None, 46, 64)            0         
                                                                 
 lstm_13 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dense_13 (Dense)            (None, 1)                

In [33]:
TEST_SIZE = 0.5
EPOCHS = 5
BATCH_SIZE = 128
#training and testing model
model.fit(tweets_preprocessed, labels, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          validation_split =TEST_SIZE)

Epoch 1/5
1047/1047 [==============================] - 208s 194ms/step - loss: 0.2322 - binary_accuracy: 0.8422 - val_loss: -39.5953 - val_binary_accuracy: 0.0000e+00
Epoch 2/5
1047/1047 [==============================] - 202s 193ms/step - loss: 7232.5728 - binary_accuracy: 0.7891 - val_loss: 8.5078 - val_binary_accuracy: 0.0000e+00
Epoch 3/5
1047/1047 [==============================] - 203s 194ms/step - loss: 34427.7227 - binary_accuracy: 0.7868 - val_loss: -74.4831 - val_binary_accuracy: 0.0000e+00
Epoch 4/5
1047/1047 [==============================] - 203s 194ms/step - loss: 10071.4160 - binary_accuracy: 0.7357 - val_loss: -139.0674 - val_binary_accuracy: 0.0000e+00
Epoch 5/5
1047/1047 [==============================] - 203s 194ms/step - loss: 0.3704 - binary_accuracy: 0.7107 - val_loss: -571.0383 - val_binary_accuracy: 0.0000e+00


In [28]:
#trying to see what the model outputs for different tweets
for x in range(4990,5010):
  tweet_ex = np.array([tweets_preprocessed[x]])
  value = model.predict(tweet_ex)
  print(value[0])
  print (labels[x])

[0.]
0
[0.]
0
[0.]
0
[2.184199e-24]
0
[0.]
0
[0.]
0
[0.]
0
[5.8406654e-19]
0
[3.4593678e-36]
0
[0.]
0
[0.]
0
[0.]
4
[0.]
4
[0.]
4
[0.]
4
[8.380916e-38]
4
[0.]
4
[0.]
4
[0.]
4
[0.]
4
